# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use exchange-specific
characterization factors in the characterization matrix of `bw2calc`.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**

# Use case with [brightway](https://brightway.dev/)

`brightway` is an open source LCA framework for Python.
To use `edges` with `brightway`, it requires that you have an activated `brightway` project with a `biosphere` database. Here, we will also use an [ecoinvent](https://ecoinvent.prg) v.3 cut-off  database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

## Example 1: simple biosphere-technosphere LCIA

In [ ]:
from edges import EdgeLCIA, get_available_methods, setup_package_logging
import bw2data

Let's make sure we have a project to work with...

In [ ]:
if "ecoinvent-3.11-cutoff" not in bw2data.projects:
    bw2io.restore_project_directory("/srv/data/brightway2-project-ecoinvent-3.11-cutoff-backup13-October-2025-04-29PM.tar.gz")

# activate the bw project
bw2data.projects.set_current("ecoinvent-3.11-cutoff")

One can simply build its own LCIA file.
Let's consider the following LCIA file (saved under `lcia_example_1.json`):

In [ ]:
method = {
  "name": "Example LCIA Method",
    "version": "1.0",
    "description": "Example LCIA method for greenhouse gas emissions",
    "unit": "kg CO2e",
    "strategies": [
        "map_exchanges"
    ],
  "exchanges": [
    {
      "supplier": {
        "name": "Carbon dioxide",
        "operator": "startswith",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
      },
      "value": 1.0
    },
    {
      "supplier": {
        "name": "Methane, fossil",
        "operator": "contains",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
      },
      "value": 28.0
    },
    {
      "supplier": {
        "name": "Dinitrogen monoxide",
        "operator": "equals",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
      },
      "value": 265.0
    }
  ]
}

Let's pick a random activity

In [ ]:
act = bw2data.Database("ecoinvent-3.11-cutoff").random()
act

In [ ]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=method,
)
LCA.lci()

LCA.apply_strategies()
#LCA.map_exchanges()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

We can print some statistics:

In [ ]:
LCA.statistics()

### Generate dataframe of characterization factors used

The `generate_cf_table` method generates a dataframe of the characterization factors used in the calculation. One can see the characterization factors used for each exchange in the system.

In [ ]:
df = LCA.generate_cf_table(include_unmatched=False)

In [ ]:
# we can see under the "CF" column
# the characterization factors used for each exchange in the system
df

In [ ]:
df.groupby("supplier name")["CF"].mean()

## Example 2: biosphere-technosphere LCIA with parameters and scenarios

In this example, the evaluation of the CFs for `methane` and `dinitrogen monoxide` is a **symbolic expression depending on the value of the parameter `co2ppm`**. It is a simple approximation of the GWP100 factor of these gases as a function of the atmospheric CO₂ concentration.

In [ ]:
method = {
  "name": "Example LCIA Method",
    "version": "1.0",
    "description": "Example LCIA method for greenhouse gas emissions",
    "unit": "kg CO2e",
    "exchanges": [
      {
        "supplier": {
          "name": "Carbon dioxide",
          "operator": "startswith",
          "matrix": "biosphere"
        },
        "consumer": {
          "matrix": "technosphere",
        },
        "value": "1.0"
      },
      {
        "supplier": {
          "name": "Methane, fossil",
          "operator": "contains",
          "matrix": "biosphere"
        },
        "consumer": {
          "matrix": "technosphere",
        },
        "value": "28 * (1 + 0.001 * (co2ppm - 410))"
      },
      {
        "supplier": {
          "name": "Dinitrogen monoxide",
          "operator": "equals",
          "matrix": "biosphere"
        },
        "consumer": {
          "matrix": "technosphere",
        },
        "value": "265 * (1 + 0.0005 * (co2ppm - 410))"
      }
  ]
}

In [ ]:
import bw2data
from edges import EdgeLCIA

# Select an activity from the LCA database
act = bw2data.Database("ecoinvent-3.11-cutoff").random()
print(act)

# Define scenario parameters (e.g., atmospheric CO₂ concentration and time horizon)
params = {
    "some scenario": {
         "co2ppm": {
             "2020": 410,
             "2050": 450,
             "2100": 500},
        "h": {
            "2020": 100,
            "2050": 100,
            "2100": 100
        }
    }
}

# Initialize LCIA
lcia = EdgeLCIA(
   demand={act: 1},
   method=method,
   parameters=params
)

# Perform inventory calculations (once)
lcia.lci()

# Map exchanges to CF entries (once)
lcia.map_exchanges()


# Run scenarios efficiently
results = []
for idx in {"2020", "2050", "2100"}:
    lcia.evaluate_cfs(idx)
    lcia.lcia()
    #df = lcia.generate_cf_table()

    #scenario_result = {
    #    "scenario": idx,
    #    "co2ppm": params["some scenario"]["co2ppm"][idx],
    #    "score": lcia.score,
    #    "CF_table": df
    #}
    #results.append(scenario_result)

    print(f"Scenario (CO₂ {params['some scenario']['co2ppm'][idx]} ppm): Impact = {lcia.score}")

## Example 3: biosphere-technosphere LCIA with function call

In this example, the evaluation of the CFs for `methane` and `dinitrogen monoxide` is returned by a call to an external function `GWP`.

We first create a function that will claculate the GWP of a given over a given tine horizon.
Inspired from the IPCC AR5, 2021.

In [ ]:
import numpy as np
# Physical constants
M_atm = 5.15e18  # kg, total mass of Earth's atmosphere
M_air = 28.96    # g/mol, average molar mass of air

# Gas-specific molecular weights (g/mol)
M_gas = {
    'CO2': 44.01,
    'CH4': 16.04,
    'N2O': 44.013
}

# IPCC concentration parameters (Myhre et al. 1998 / IPCC AR6)
RF_COEFF = {
    'CH4': 0.036,  # W·m⁻²·ppb⁻½ for CH4
    'N2O': 0.12    # W·m⁻²·ppb⁻½ for N2O
}

# Reference atmospheric concentrations (IPCC AR6, ~2019)
C_REF = {
    'CH4': 1866,  # ppb
    'N2O': 332    # ppb
}

# Indirect forcing factor for methane (IPCC AR6)
INDIRECT_FACTOR = {
    'CH4': 1.65,
    'N2O': 1.0
}

# Gas-specific atmospheric lifetimes (years, IPCC AR6)
TAU_GAS = {
    'CH4': 11.8,
    'N2O': 109
}

# CO2 impulse response function parameters (IPCC AR5/AR6)
CO2_IRF = {
    'a0': 0.2173,
    'a': [0.2240, 0.2824, 0.2763],
    'tau': [394.4, 36.54, 4.304]
}

# Convert concentration-based radiative efficiency to mass-based (W·m⁻²·kg⁻¹)
def convert_ppb_to_mass_rf(a_ppb, gas):
    return a_ppb * (M_atm / M_gas[gas]) * (M_air / 1e9)

# Calculate concentration-dependent radiative efficiency
def radiative_efficiency_concentration(gas, concentration_ppb):
    alpha = RF_COEFF[gas]
    return (alpha / (2 * np.sqrt(concentration_ppb))) * INDIRECT_FACTOR[gas]

# AGWP for CO2 (mass-based)
def AGWP_CO2(H):
    integral_CO2 = CO2_IRF['a0'] * H + sum(
        a * tau * (1 - np.exp(-H / tau))
        for a, tau in zip(CO2_IRF['a'], CO2_IRF['tau'])
    )
    am_CO2 = convert_ppb_to_mass_rf(1.37e-5, 'CO2')  # fixed IPCC radiative efficiency for CO2
    return am_CO2 * integral_CO2

# AGWP for gas at given concentration
def AGWP_gas(gas, H, concentration_ppb):
    aa_gas = radiative_efficiency_concentration(gas, concentration_ppb)
    am_gas = convert_ppb_to_mass_rf(aa_gas, gas)
    tau_gas = TAU_GAS[gas]
    return am_gas * tau_gas * (1 - np.exp(-H / tau_gas))

# Calculate concentration-dependent GWP
def GWP(gas, H, concentration_ppb):
    AGWP_g = AGWP_gas(gas, H, concentration_ppb)
    AGWP_ref = AGWP_CO2(H)
    return AGWP_g / AGWP_ref


In [ ]:
GWP('CH4', 100, 1911)

In [ ]:
method = {
  "name": [
    "Parameterized GWP method"
  ],
  "unit": "kg CO2-eq",
  "description": "Parameterized GWP factors.",
  "version": "1.0",
  "exchanges": [
    {
      "supplier": {
        "name": "Carbon dioxide",
        "operator": "startswith",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
      },
      "value": "1.0"
    },
    {
        "supplier": {
          "name": "Methane, fossil",
          "operator": "contains",
          "matrix": "biosphere"
        },
        "consumer": {
          "matrix": "technosphere",
        },
        "value": "GWP('CH4',H, C_CH4)"
      },
    {
      "supplier": {
        "name": "Dinitrogen monoxide",
        "operator": "equals",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
      },
      "value": "GWP('N2O',H, C_N2O)"
    }
  ],
  "scenarios": {
    "RCP1.9": {
      "C_CH4": {
        "2020": 1866,
        "2050": 1428,
        "2080": 1150,
        "2100": 1036
      },
      "C_N2O": {
        "2020": 332,
        "2050": 344,
        "2080": 350,
        "2100": 354
      },
      "H": {
        "2020": 100,
        "2050": 100,
        "2080": 100,
        "2100": 100
      }
    },
    "RCP2.6": {
      "C_CH4": {
        "2020": 1866,
        "2050": 1519,
        "2080": 1197,
        "2100": 1056
      },
      "C_N2O": {
        "2020": 332,
        "2050": 344,
        "2080": 349,
        "2100": 354
      },
      "H": {
        "2020": 100,
        "2050": 100,
        "2080": 100,
        "2100": 100
      }
    },
    "RCP4.5": {
      "C_CH4": {
        "2020": 1866,
        "2050": 2020,
        "2080": 1779,
        "2100": 1683
      },
      "C_N2O": {
        "2020": 332,
        "2050": 356,
        "2080": 373,
        "2100": 377
      },
      "H": {
        "2020": 100,
        "2050": 100,
        "2080": 100,
        "2100": 100
      }
    },
    "RCP8.5": {
      "C_CH4": {
        "2020": 1866,
        "2050": 2446,
        "2080": 2652,
        "2100": 2415
      },
      "C_N2O": {
        "2020": 332,
        "2050": 358,
        "2080": 380,
        "2100": 392
      },
      "H": {
        "2020": 100,
        "2050": 100,
        "2080": 100,
        "2100": 100
      }
    }
  }
}



In [ ]:
# we declare the function
allowed_funcs = {"GWP": GWP}

In [ ]:
all_results = []  # To collect results

# Initialize LCIA
lcia = EdgeLCIA(
    demand={act: 1},
    method=method,
    allowed_functions=allowed_funcs # we declare the functions to call.
)

# Usual LCIA steps
lcia.lci()
lcia.map_exchanges()

results = []
# Loop over scenarios
for scenario in [
    "RCP1.9",
    "RCP2.6",
    "RCP4.5",
    "RCP8.5",
]:
    for year in [
        "2020",
        "2050",
        "2080",
        "2100",
    ]:
        print(scenario, year)
        lcia.evaluate_cfs(scenario=scenario, scenario_idx=year)
        lcia.lcia()
        # df = lcia.generate_cf_table()

        #scenario_result = {
        #    "year": int(year),
        #    "rcp": scenario,
        #    "score": lcia.score,
        #    "CF_table": df
        #}
        #all_results.append(scenario_result)

        print(f"Year {year} | {scenario}: Impact = {lcia.score}")

## Example 5: biosphere-technosphere LCIA with geographies and uncertainty

In this example, we try to characterize water withdrawals, and uncertainty information.

In [ ]:
from edges import EdgeLCIA, get_available_methods, setup_package_logging
import bw2data

In [ ]:
method = {
  "name": "Example LCIA Method",
    "version": "1.0",
    "description": "Example LCIA method for water withdrawals",
    "unit": "m3 freshwater",
    "strategies": [
        "map_exchanges",
        "map_contained_locations"
    ],
  "exchanges": [
    {
         "supplier": {
            "name": "Water",
            "matrix": "biosphere"
         },
         "consumer": {
            "location": "RER",
            "matrix": "technosphere",
         },
         "value": 17,
         "weight": 213938000,
         "uncertainty": {
            "distribution": "discrete_empirical",
            "parameters": {
               "values": [
                  6.87,
                  7.56,
                  13.4,
                  15.5,
                  18.7,
                  19.1,
                  21.5,
                  21.8,
                  27.6,
                  29.8,
                  31.0,
                  33.5,
                  33.7,
                  34.6
               ],
               "weights": [
                  0.153,
                  0.324,
                  0.015,
                  0.038,
                  0.079,
                  0.008,
                  0.0,
                  0.075,
                  0.074,
                  0.0,
                  0.008,
                  0.013,
                  0.08,
                  0.131
               ]
            }
         }
      },
      {
         "supplier": {
            "name": "Water",
            "matrix": "biosphere"
         },
         "consumer": {
            "location": "FR",
            "matrix": "technosphere",
            "classifications": {
               "CPC": [
                  "01"
               ]
            }
         },
         "value": 10,
         "weight": 213938000,
         "uncertainty": {
            "distribution": "triang",
            "parameters": {
               "loc": 11,
               "minimum": 0.752,
               "maximum": 50.6
            }
         }
      }
  ]
}

In [ ]:
act = bw2data.Database("ecoinvent-3.11-cutoff").random()
act

In [ ]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=method,
)
LCA.lci()

LCA.apply_strategies()
#LCA.map_exchanges()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

In [ ]:
df = LCA.generate_cf_table()

In [ ]:
df

In [ ]:
df.groupby("consumer location")["CF"].unique()

In [ ]:
df.loc[
    (df["consumer location"] == "FR")
]

In [ ]:
act = [
    x for x in bw2data.Database("ecoinvent-3.11-cutoff") 
    if x['location']=='FR' and "electricity production, hydro" in x["name"]
][0]
print(act)
LCA = EdgeLCIA(
    demand={act: 1},
    method=method,
    use_distributions=True,
    iterations=10_000
)
LCA.lci()

LCA.apply_strategies()
#LCA.map_exchanges()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

In [ ]:
type(LCA.score.data)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(LCA.score.data, bins=50)

In [ ]:
df = LCA.generate_cf_table()

In [ ]:
df

## Example 6: technosphere-technosphere LCIA

In this example, we want to characterize intermediate product exchanges.
Here, we are interested in the amount of **secondary copper** flowing in the system.

In [ ]:
from edges import EdgeLCIA, get_available_methods, setup_package_logging
import bw2data

In [ ]:
method = {
  "name": "ecoinvent 3.10/3.11 - RELICS - Copper, secondary",
  "unit": "kg",
  "version": "1.0",
  "strategies": [
       "map_exchanges"
   ],
  "exchanges": [
    {
      "supplier": {
        "name": "treatment of",
        "reference product": "copper, cathode",
        "operator": "startswith",
        "matrix": "technosphere",
        "excludes": [
          "market"
        ]
      },
      "consumer": {
        "matrix": "technosphere"
      },
      "value": 1.0
    },
    {
        "supplier": {
          "name": "treatment of",
          "reference product": "copper, anode",
          "operator": "startswith",
          "matrix": "technosphere",
          "excludes": [
            "market",
            "slime"
          ]
        },
        "consumer": {
          "matrix": "technosphere"
        },
        "value": 1.0
      }
  ]
}

In [ ]:
act = bw2data.Database("ecoinvent-3.11-cutoff").random()
act

In [ ]:
LCA = EdgeLCIA(
    demand={act: 1000},
    method=method,
)
LCA.lci()

LCA.apply_strategies()
#LCA.map_exchanges()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

In [ ]:
df = LCA.generate_cf_table()

In [ ]:
df

### Bonus: Sankey diagram

In [ ]:
from edges import SupplyChain

In [ ]:
sc = SupplyChain(
    activity=act,
    method=method,
    amount=1,
    level=5,
    cutoff=0.01,
    cutoff_basis="total",
    collapse_markets=False,
)
total = sc.bootstrap()
df, total_score, ref_amount = sc.calculate()

sc.save_html(
    df,
    path="example_sankey.html",
    height_max=1000,
    width_max=2400,
    node_instance_mode="by_parent",  # or "by_child_level" / "by_level"
    node_thickness=12
)